<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 828ms/step - loss: 0.6722 - accuracy: 0.5949 - val_loss: 0.6987 - val_accuracy: 0.4732
Epoch 2/10
26/26 [==============================] - 20s 784ms/step - loss: 0.6324 - accuracy: 0.6630 - val_loss: 0.6314 - val_accuracy: 0.6000
Epoch 3/10
26/26 [==============================] - 20s 765ms/step - loss: 0.6073 - accuracy: 0.6338 - val_loss: 0.5569 - val_accuracy: 0.7512
Epoch 4/10
26/26 [==============================] - 20s 773ms/step - loss: 0.5569 - accuracy: 0.7202 - val_loss: 0.5094 - val_accuracy: 0.8537
Epoch 5/10
26/26 [==============================] - 20s 761ms/step - loss: 0.4985 - accuracy: 0.7640 - val_loss: 0.4605 - val_accuracy: 0.7756
Epoch 6/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4166 - accuracy: 0.8601 - val_loss: 0.3554 - val_accuracy: 0.9171
Epoch 7/10
26/26 [==============================] - 20s 762ms/step - loss: 0.3909 - accuracy: 0.8394 - val_loss: 0.3021 - val_accuracy: 0.9317

In [13]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1271), started 0:00:28 ago. (Use '!kill 1271' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 20s - loss: 0.6663 - accuracy: 0.5888 - val_loss: 0.6790 - val_accuracy: 0.5073
Epoch 2/5

Epoch 00002: saving model to weights.02-0.58.h5
26/26 - 20s - loss: 0.6125 - accuracy: 0.6910 - val_loss: 0.5812 - val_accuracy: 0.7854
Epoch 3/5

Epoch 00003: saving model to weights.03-0.57.h5
26/26 - 19s - loss: 0.5759 - accuracy: 0.7226 - val_loss: 0.5720 - val_accuracy: 0.6927
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 19s - loss: 0.5366 - accuracy: 0.7579 - val_loss: 0.5341 - val_accuracy: 0.7463
Epoch 5/5

Epoch 00005: saving model to weights.05-0.42.h5
26/26 - 19s - loss: 0.4623 - accuracy: 0.8139 - val_loss: 0.4162 - val_accuracy: 0.8585


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 20s - loss: 0.6722 - accuracy: 0.6217 - val_loss: 0.6664 - val_accuracy: 0.5171


In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6752 - accuracy: 0.5730 - val_loss: 0.6641 - val_accuracy: 0.7220
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 19s - loss: 0.6488 - accuracy: 0.6679 - val_loss: 0.6745 - val_accuracy: 0.5024


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6683 - accuracy: 0.6217 - val_loss: 0.7178 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 19s - loss: 0.6204 - accuracy: 0.6825 - val_loss: 0.5915 - val_accuracy: 0.7366
Epoch 3/50
26/26 - 19s - loss: 0.5650 - accuracy: 0.7275 - val_loss: 0.6146 - val_accuracy: 0.6049
Epoch 4/50
26/26 - 19s - loss: 0.5138 - accuracy: 0.7555 - val_loss: 0.4698 - val_accuracy: 0.7707
Epoch 5/50
26/26 - 19s - loss: 0.4568 - accuracy: 0.8005 - val_loss: 0.4239 - val_accuracy: 0.8244
Epoch 6/50
26/26 - 19s - loss: 0.4128 - accuracy: 0.8248 - val_loss: 0.4679 - val_accuracy: 0.7805
Epoch 7/50
26/26 - 19s - loss: 0.3532 - accuracy: 0.8528 - val_loss: 0.3544 - val_accuracy: 0.8780
Epoch 8/50
26/26 - 19s - loss: 0.3281 - accuracy: 0.8796 - val_loss: 0.2791 - val_accuracy: 0.8878
Epoch 9/50
26/26 - 20s - loss: 0.2531 - accuracy: 0.9282 - val_loss: 0.2234 - val_accuracy: 0.9122
Epoch 10/50
26/26 - 19s - loss: 0.2380 - accuracy: 0.9161 - val_loss: 0.2661 - val_accuracy: 0.8732
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 806ms/step - loss: 0.6704 - accuracy: 0.5815 - val_loss: 0.6535 - val_accuracy: 0.6049
Epoch 2/5
26/26 [==============================] - 20s 759ms/step - loss: 0.6266 - accuracy: 0.6484 - val_loss: 0.7343 - val_accuracy: 0.5024
Epoch 3/5
26/26 [==============================] - 20s 765ms/step - loss: 0.5732 - accuracy: 0.7275 - val_loss: 0.6817 - val_accuracy: 0.5415
Epoch 4/5
26/26 [==============================] - 20s 751ms/step - loss: 0.5273 - accuracy: 0.7482 - val_loss: 0.5444 - val_accuracy: 0.7317
Epoch 5/5
26/26 [==============================] - 19s 749ms/step - loss: 0.5077 - accuracy: 0.7518 - val_loss: 0.5644 - val_accuracy: 0.6780


In [15]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.581508,0.669951,0.604878,0.653483
1,1,0.648419,0.627254,0.502439,0.734311
2,2,0.727494,0.574621,0.541463,0.681730
3,3,0.748175,0.526024,0.731707,0.544382
4,4,0.751825,0.507140,0.678049,0.564403


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 21s 803ms/step - loss: 0.6756 - accuracy: 0.5912 - val_loss: 0.6865 - val_accuracy: 0.4537

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 761ms/step - loss: 0.6502 - accuracy: 0.6253 - val_loss: 0.6599 - val_accuracy: 0.5707

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6367 - accuracy: 0.6703 - val_loss: 0.6474 - val_accuracy: 0.7122

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 754ms/step - loss: 0.6293 - accuracy: 0.7543 - val_loss: 0.6439 - val_accuracy: 0.7073

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 768ms/step - loss: 0.6

In [17]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 833ms/step - loss: 0.6844 - accuracy: 0.5474 - val_loss: 0.6993 - val_accuracy: 0.4634
Epoch 2/50
26/26 [==============================] - 20s 781ms/step - loss: 0.6488 - accuracy: 0.6265 - val_loss: 0.6195 - val_accuracy: 0.8098
Epoch 3/50
26/26 [==============================] - 20s 773ms/step - loss: 0.5873 - accuracy: 0.7372 - val_loss: 0.5685 - val_accuracy: 0.6683
Epoch 4/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5611 - accuracy: 0.7188
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 769ms/step - loss: 0.5595 - accuracy: 0.7202 - val_loss: 0.5898 - val_accuracy: 0.6098
Epoch 5/50
26/26 [==============================] - 20s 773ms/step - loss: 0.4929 - accuracy: 0.8175 - val_loss: 0.4841 - val_accuracy: 0.8488
Epoch 6/50
26/26 [==============================] - 20s 770ms/step - loss: 0.4725 - accuracy: 0.8309 - val_loss: 0.4751 - 

In [ ]:
%tensorboard --logdir log_dir